In [24]:
# Import dependencies
import os
import subprocess
import json
from dotenv import load_dotenv
import pprint
from bit import PrivateKeyTestnet

pp = pprint.PrettyPrinter(indent=2)

from eth_account import Account

from bit.network import NetworkAPI

In [25]:
# Load and set environment variables
load_dotenv("/Users/FinTech/PycharmProjects/Bootcamp/HW/19-Wallet/mnemonic_keys.env")
mnemonic = os.getenv("mnemonic")
mnemonic

'tomato idea borrow mixture admit pyramid flight dumb wall torch egg disease'

In [26]:
# Import constants.py and necessary functions from bit and web3
from constants import *
from web3 import Web3
from web3.middleware import geth_poa_middleware

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [27]:
# Create a function called `derive_wallets`
def derive_wallets(coinType):
    command = 'php hd-wallet-derive/hd-wallet-derive.php ' \
              '-g ' \
              '--mnemonic=mnemonic ' \
              '--cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub ' \
              '--format=json ' \
              '--coin=' + coinType + ' ' \
                                     '--numderive=3'
    print('Command to execute: ' + command)
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=False)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [28]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {ETH: derive_wallets(ETH), BTCTEST: derive_wallets(BTCTEST)}
pp.pprint(coins)
#print("Private key of the 2nd ethereum index is: " + coins[ETH][1]['privkey'])

Command to execute: php hd-wallet-derive/hd-wallet-derive.php -g --mnemonic=mnemonic --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=json --coin=eth --numderive=3
Command to execute: php hd-wallet-derive/hd-wallet-derive.php -g --mnemonic=mnemonic --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --format=json --coin=btc-test --numderive=3
{ 'btc-test': [ { 'address': 'mtgb9hthmFpY1zidrYmZ24eTF87aBA147h',
                  'index': 0,
                  'path': "m/44'/1'/0'/0/0",
                  'privkey': 'cVAgDoahzo5Ucdv8ioxxubvmE6LfU3ZQgJ21UvY4eC1xi6WkvnXe',
                  'pubkey': '0339b281b12687c70a024ef409229b55d404533b4e9a475dc9d3883c0172913fb6',
                  'pubkeyhash': '906c250da82c2bd3edf9bcbbbb29f2188065f9d2',
                  'xprv': 'tprv8kmimjp9GYq9EvcrxozEhxzk9jGwaC73BSiy3ATqwwikM4YgH7685zQLzX7xzXAmhXxf5LZbpbqtSGX3fcNFbfVJdxuEYPrA5tDRRQgwPRm',
                  'xpub': 'tpubDHTkv9rPQvWp8PeerTeq7NeriknsjXHwkkKkKgW9NDX9BYoSuV

In [29]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, private_key):
    if coin == 'eth':
        account_eth = Account.from_key(private_key)
        return account_eth
    if coin == 'btc-test':
        account_btctest = PrivateKeyTestnet(private_key)
        return account_btctest

#print('btc address:' + ' ' + priv_key_to_account('btc-test', "cVAgDoahzo5Ucdv8ioxxubvmE6LfU3ZQgJ21UvY4eC1xi6WkvnXe").address)
#print('eth address:' + ' ' + priv_key_to_account('eth', "0x82d63adc8be924a92ef77c6e3286c4e80eaebad63ba2b91c27801a62ba2f9410").address)

#account_btctest = priv_key_to_account('btc-test', "cVAgDoahzo5Ucdv8ioxxubvmE6LfU3ZQgJ21UvY4eC1xi6WkvnXe")
#account_eth = priv_key_to_account('eth', "0x82d63adc8be924a92ef77c6e3286c4e80eaebad63ba2b91c27801a62ba2f9410")

#account_btctest

In [30]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, recipient, amount):
    if coin == 'eth':
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        return {
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
    if coin == 'btc-test':
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, 'btc')])

#create_tx('btc-test', account_btctest, 'mxwSU31v1NtbUi6Pc5bN9BgmmAGw13JdvX', 0.0005)

In [31]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, recipient, amount):
    if coin == 'eth':
        tx = create_tx(coin, account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    if coin == 'btc-test':
        tx = create_tx(coin, account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        return NetworkAPI.broadcast_tx_testnet(signed_tx)
    
#send_tx('btc-test', account_btctest, 'mxwSU31v1NtbUi6Pc5bN9BgmmAGw13JdvX', 0.0005)

In [54]:
account_btctest = priv_key_to_account('btc-test', "cVAgDoahzo5Ucdv8ioxxubvmE6LfU3ZQgJ21UvY4eC1xi6WkvnXe")

account_btctest.get_balance()

'161352'

In [32]:
print(dir(Account))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_default_kdf', '_keys', '_parsePrivateKey', '_recover_hash', '_sign_hash', '_use_unaudited_hdwallet_features', 'create', 'create_with_mnemonic', 'decrypt', 'enable_unaudited_hdwallet_features', 'encrypt', 'from_key', 'from_mnemonic', 'privateKeyToAccount', 'recoverHash', 'recoverTransaction', 'recover_message', 'recover_transaction', 'setKeyBackend', 'set_key_backend', 'signHash', 'signTransaction', 'sign_message', 'sign_transaction']


In [33]:
private_key = "cVAgDoahzo5Ucdv8ioxxubvmE6LfU3ZQgJ21UvY4eC1xi6WkvnXe"

In [34]:
account = PrivateKeyTestnet(private_key)

In [35]:
account.address

'mtgb9hthmFpY1zidrYmZ24eTF87aBA147h'

In [36]:
account.get_balance()

'200000'

In [37]:
account = priv_key_to_account('btc-test', private_key)
account.address

'mtgb9hthmFpY1zidrYmZ24eTF87aBA147h'

In [38]:
send_tx('btc-test', account, 'mxwSU31v1NtbUi6Pc5bN9BgmmAGw13JdvX', 0.000005)

In [39]:
account_two = priv_key_to_account('btc-test', "cRG5vrAkkq3ZkTmT2mmqCgbxd8niBdXx3ocDMd1771oSzJDvqXiC")

In [43]:
account_two.get_balance()

'500'

In [41]:
account.get_transactions()

['c9d609d2c98d69c84e8118e6c852f8bb73b9ec3f342029eff01cbccb90822a43',
 'd4cad5b109855e58361fc77863ac184d11298ee142d13909c67a7598c7bc936a',
 'd99c7d37f01426755bdce75e0476691691d7fe2dc837cd16729ea417fdeeca14',
 'f3e3bdefb7f9fe671f87bde0267b6d7411ec9321db67b0864530bef4e809f10d',
 '0b7473eff1b65868e117048884117f19722d9ac340f3d6afacd58980da6aa693']

In [44]:
account.get_balance()

'161352'